In [1]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt


landmarks_color_mapping = {
    "eyeLeft": "#00FFFF", "eyeRight": "#F5F5DC", "mouthLeft": "#FFEBCD",
    "mouthRight": "#A52A2A", "nose": "#7FFF00", "leftEyeBrowLeft": "#6495ED",
    "leftEyeBrowRight": "#00FFFF", "leftEyeBrowUp": "#B8860B", "rightEyeBrowLeft": "#BDB76B", 
    "rightEyeBrowRight": "#FF8C00", "rightEyeBrowUp": "#E9967A", "leftEyeLeft": "#2F4F4F",
    "leftEyeRight": "#FF1493", "leftEyeUp": "#1E90FF", "leftEyeDown": "#228B22",
    "rightEyeLeft": "#F8F8FF", "rightEyeRight": "#808080", "rightEyeUp": "#F0FFF0",
    "rightEyeDown": "#4B0082", "noseLeft": "#E6E6FA", "noseRight": "#FFFACD",
    "mouthUp": "#E0FFFF", "mouthDown": "#D3D3D3", "leftPupil": "#20B2AA",
    "rightPupil": "#B0C4DE", "upperJawlineLeft": "#32CD32", "midJawlineLeft": "#800000",
    "chinBottom": "#BA55D3", "midJawlineRight": "#7B68EE", "upperJawlineRight": "#C71585"
}

frames_root_dir = './raw_volvo_demo_video_frames/'
face_detection_results_root_dir = './face_detection_results/'

frames = [ele for ele in os.listdir(frames_root_dir) if ele.endswith('.jpg')]
frames = sorted(frames, key=lambda x: int(x.split('.jpg')[0]))

output_frames_root_dir = './output/frames/'
if not os.path.exists(output_frames_root_dir):
    os.makedirs(output_frames_root_dir)

for frame_name in frames[4090:]:
    print('Processing frame {}'.format(frame_name))

    image = cv2.imread(os.path.join(frames_root_dir, frame_name), cv2.IMREAD_COLOR)
    image_height = image.shape[0]
    image_width = image.shape[1]
    
    with open(os.path.join(face_detection_results_root_dir, frame_name.split('.jpg')[0] + '.json'), 'r') as rf:
        response = json.load(rf)
    
    face_details = response['FaceDetails']

    fig = plt.figure(figsize=(19.2, 10.8))
    ax = fig.add_subplot(111)
    plt.imshow(image[:, :, ::-1])
    plt.axis('off')
    texts = list()
            
    # draw bounding boxes and landmarks
    for face_index in range(len(face_details)):
        height = int(face_details[face_index]['BoundingBox']['Height'] * image_height)
        left = int(face_details[face_index]['BoundingBox']['Left'] * image_width)
        top = int(face_details[face_index]['BoundingBox']['Top'] * image_height)
        width = int(face_details[face_index]['BoundingBox']['Width'] * image_width)
        
        # get attributes
        face_prob = round(face_details[face_index]['Confidence'], 2)
        gender = face_details[face_index]['Gender']['Value']
        age_low = face_details[face_index]['AgeRange']['Low']
        age_high = face_details[face_index]['AgeRange']['High']
        emo_list = face_details[face_index]['Emotions']
        emo_list = sorted(emo_list, key=lambda x: -x['Confidence'])
        emotion = emo_list[0]['Type']
        wearglasses = 'Yes' if face_details[face_index]['Sunglasses']['Value'] is True or face_details[face_index]['Eyeglasses']['Value'] is True else 'No'
        mustache = 'Yes' if face_details[face_index]['Mustache']['Value'] is True else 'No'
        beard = 'Yes' if face_details[face_index]['Beard']['Value'] is True else 'No'
        pitch = round(face_details[face_index]['Pose']['Pitch'], 1)
        roll = round(face_details[face_index]['Pose']['Roll'], 1)
        yaw = round(face_details[face_index]['Pose']['Yaw'], 1)

        # (left, top)
        # (left + width, top + height)
        x_min, y_min, x_max, y_max = left, top, left + width, top + height
        x_min = 0 if x_min < 0 else x_min
        x_min = image_width - 1 if x_min >= image_width else x_min
        y_min = 0 if y_min < 0 else y_min
        y_min = image_height - 1 if y_min >= image_height else y_min
        
        x_max = 0 if x_max < 0 else x_max
        x_max = image_width - 1 if x_max >= image_width else x_max
        y_max = 0 if y_max < 0 else y_max
        y_max = image_height - 1 if y_max >= image_height else y_max
        
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, fill=True,
                          edgecolor=None, linewidth=1, facecolor='#FFFFE0', alpha=0.10)
        ax.add_patch(rect)
        
        if gender == 'Male':
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, color='#7FFFD4', fill=False, linewidth=1.5)
            ax.add_patch(rect)
        else:
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, color='#DB7093', fill=False, linewidth=1.5)
            ax.add_patch(rect) 
        
        # draw landmarks
        landmarks = face_details[face_index]['Landmarks']
        for mark in landmarks:
            x = int(mark['X'] * image_width)
            y = int(mark['Y'] * image_height)
            point_type = mark['Type']
            if x >= 3 and x < image_width - 3 and y >= 3 and y < image_height:
                plt.plot(x, y, '.', color=landmarks_color_mapping[point_type])
        
        # place text to describe the face attributes  
        text_position = None

        text_area_candidates = [
            [left + width + 8, top, left + width + 8 + 210, top + 210],  # 右侧顶端对齐
            [left - 200, top, left, top + 210],  # 左侧顶端对齐
            [left, top + height + 8, left + 210, top + height + 8 + 210],# 下侧左端对齐
            [left, top - 200, left + 210, top],  # 上侧左端对齐
            [left + width + 8, top + height + 8, left + width + 8 + 210, top + height + 8 + 210], # 右下侧左端对齐
            [left - 200, top + height + 8, left, top + height + 8 + 210], # 左下侧右侧对齐
            [left - 200, top - 210, left, top], # 左上侧右侧（底部）对齐
            [left + width + 8, top - 210, left + width + 8 + 210, top] #右上侧对齐
        ]
        
        margin = 40
        for area in text_area_candidates:
            if area[0] > margin and area[1] > margin and area[2] < image_width - margin and area[3] < image_height - margin:
                text_position = area
                break
    
        if text_position is None:
            continue
        
        plt.text(text_position[0], text_position[1], 
                 "-Face Prob: {}%\n-Gender: {}\n-Age: {}~{} years old\n-Emotion: {}\n-Wear Glasses: {}\n-Mustache: {}\n-Beard: {}\n-Pose: {}/{}/{}".format(
                     face_prob,
                     gender,
                     age_low, age_high,
                     emotion,
                     wearglasses,
                     mustache,
                     beard,
                     pitch,
                     roll,
                     yaw
                 ), 
                 size = 8, color = "#FFD700", weight = 'bold', ha = 'left', va = 'top', in_layout=True,
                 bbox = dict(boxstyle = 'round', facecolor = "#FF00FF", alpha = 0.2))
    
    fig.savefig(os.path.join(output_frames_root_dir, frame_name),
                transparent=True, dpi=300, pad_inches=0, bbox_inches='tight')
    plt.close()

Processing frame 4090.jpg
Processing frame 4091.jpg
Processing frame 4092.jpg
Processing frame 4093.jpg
Processing frame 4094.jpg
Processing frame 4095.jpg
Processing frame 4096.jpg
Processing frame 4097.jpg
Processing frame 4098.jpg
Processing frame 4099.jpg
Processing frame 4100.jpg
Processing frame 4101.jpg
Processing frame 4102.jpg
Processing frame 4103.jpg
Processing frame 4104.jpg
Processing frame 4105.jpg
Processing frame 4106.jpg
Processing frame 4107.jpg
Processing frame 4108.jpg
Processing frame 4109.jpg
Processing frame 4110.jpg
Processing frame 4111.jpg
Processing frame 4112.jpg
Processing frame 4113.jpg
Processing frame 4114.jpg
Processing frame 4115.jpg
Processing frame 4116.jpg
Processing frame 4117.jpg
Processing frame 4118.jpg
Processing frame 4119.jpg
Processing frame 4120.jpg
Processing frame 4121.jpg
Processing frame 4122.jpg
Processing frame 4123.jpg
Processing frame 4124.jpg
Processing frame 4125.jpg
Processing frame 4126.jpg
Processing frame 4127.jpg
Processing f

Processing frame 4406.jpg
Processing frame 4407.jpg
Processing frame 4408.jpg
Processing frame 4409.jpg
Processing frame 4410.jpg
Processing frame 4411.jpg
Processing frame 4412.jpg
Processing frame 4413.jpg
Processing frame 4414.jpg
Processing frame 4415.jpg
Processing frame 4416.jpg
Processing frame 4417.jpg
Processing frame 4418.jpg
Processing frame 4419.jpg
Processing frame 4420.jpg
Processing frame 4421.jpg
Processing frame 4422.jpg
Processing frame 4423.jpg
Processing frame 4424.jpg
Processing frame 4425.jpg
Processing frame 4426.jpg
Processing frame 4427.jpg
Processing frame 4428.jpg
Processing frame 4429.jpg
Processing frame 4430.jpg
Processing frame 4431.jpg
Processing frame 4432.jpg
Processing frame 4433.jpg
Processing frame 4434.jpg
Processing frame 4435.jpg
Processing frame 4436.jpg
Processing frame 4437.jpg
Processing frame 4438.jpg
Processing frame 4439.jpg
Processing frame 4440.jpg
Processing frame 4441.jpg
Processing frame 4442.jpg
Processing frame 4443.jpg
Processing f